#                                           Whoosh

Whoosh is a full-featured text search engine library in Python. It offers fast indexing and retrieval compared to other pure-Python, scoring and full-text search solutions. Programmers can use it to easily add search functionality to their applications and websites. A wide variety of scoring algorithms can be used with Whoosh. The default scoring algorithm is BM-25. One requirement is that you have to specify the fields on which you want to index. Whoosh provides a lot of features for search, like, boosting the score of results which match a specified field, fuzzy search, stemming, search on combinations of query words etc. It does not need external dependencies and is open to customization in order to add additional functionalities.

Whoosh was created and is maintained by Matt Chaput. It was originally created for use in the online help system of Side Effects Software’s 3D animation software Houdini and later the code was open-sourced.

## Other Tools?
There are several other applications like Solr (an open source enterprise search server based on Lucene search library), ElasticSearch (a distributed, RESTful, free/open source search server based on Java under the Apache License-Lucene), Lucene etc all of which are written in Java. Though Lucene has a Python wrapper called pylucene, the simplicity, ease in understanding and "pure-Pythoness" favors Whoosh compared to the complex capabilities and functionalities other packages offer.
Scout is a RESTful search server written in Python which can be used as an alternative to Whoosh when you have limited search requirement, like exact keyword match. The option to specify the schema is not available in Scout and hence search is executed on only one field. But it is easy to setup and maintain.

## Installation
!pip3 install Whoosh or
easy_install Whoosh

### Getting the source

Download source releases from PyPI at http://pypi.python.org/pypi/Whoosh/

## Introduction
Whoosh was built to create an index that stored all the text and meta-data associated with each document in a dataset. While implementing search on document sets, the index can store the title, content, path of each document in the collection. Other metadata like created date, file size, keywords can also be stored. Each piece of information about the document is called a field. So, the title, content are all fields of the index and one or a set of fields can be used to search the index of documents. 

The first part of the spotlight demonstrates creating an index and searching using query words on a smaller corpus and in the second part, we'll see how to use Whoosh on a large dataset. We will also see some of the features provided by Whoosh such as fuzzy search and correcting errors in user queries.

## Part 1: Getting your feet wet with Whoosh
### Define schema
The first step is to define the schema which specifies the fields to be stored for each document. The schema is similar to that in relational databases where we specify the columns of a table and other related metadata. The schema outlines the structure of the search index, providing metadata about each of the fields that will be indexed.

Here we define a schema consisting of title of the document, the path on the filesystem where it is stored and the content of the document as the fields. Once the schema is defined, we can create an index to store the necessary documents. Some fields can be indexed, and some fields can be stored with the document so the field value is available in search results. Specifying **_stored=True_** while defining the schema exacts this requirement. The scheme has to be created only once along with indexing. We can specify a field boost for a field. This is a multiplier applied to the score of any term found in the field. We have applied a boost of 2.0 for the field title using **_field_boost_** flag here.

### Field types:
Some of the predefined field types provided by Whoosh are:
- whoosh.fields.TEXT -  This type is for body text. It indexes (and optionally stores) the text using the **_stored=True_** flag.
 
- whoosh.fields.KEYWORD - This field type is designed for space or comma-separated keywords and is indexed and searchable (and optionally stored using **_stored_** flag). By default, the keywords are space separated. To separate the keywords by commas instead, use **_commas=True_**. Use **_scorable=True_** to enable searching on the keyword fields.

- whoosh.fields.ID - The ID field type indexes (and optionally stores) the entire value of the field as a single unit. ID can be used for fields like url or path of a document, date – fields where the value must be treated as a whole, and each document only has one value for the field. By default, ID fields are not stored and can be stored using **_stored=True_** flag.

In [22]:
# import required packages
from whoosh.fields import Schema, TEXT, ID, DATETIME
from whoosh import index
import os.path

# boost the title field score twice as high as terms in the path and content fields
schema = Schema(title=TEXT(stored=True, field_boost=2.0), path=ID(stored=True), content=TEXT(stored = True))

### Create Index
Next, we create the index. First we check to see whether or not the index (as we name the folder) already exists on our machine. If it doesn't then we create it otherwise we overwrite it and structure it with our schema. 

Once we’ve created an Index object, we can add documents to the index with an IndexWriter object using  Index.writer() method. Whoosh locks out all attempts to write if there's another writer open, so we open the index writer using a with statement which causes it to automatically close/commit when it finishes. There is also the option to manually commit and close the writer after adding all the documents. Documents are indexed as per schema and has to be added as per schema design. It's not necessary to fill in a value for every field. Whoosh doesn’t care if we leave out a field from a document. Indexed fields must be passed a unicode value.

If we want to replace or re-index a document, we can delete the old document using one of the delete_* methods on Index or IndexWriter, then use IndexWriter.add_document to add the new version. Or, we can use IndexWriter.update_document to do this in one step.

In [23]:
# Check if indexDir exists and create it if it doesn't
if not os.path.exists("indexDir"):
    os.mkdir("indexDir")
    
# Create the index using the specified schema
ix = index.create_in("indexDir", schema)

# List of documents to index
documents = [{"title":u"Example", "content":u"This is an example for whoosh search", "path":u"/a", "abc":u"abc"},
            {"title":u"System", "content":u"A system is a group of interacting or interrelated entities that form a unified whole", "path":u"/b"},
            {"title":u"System example", "content":u"An example of a system is all the organs that work together for digestion", "path":u"/c"},
            {"title":u"Types of systems", "content":u"There are two major types: Natural Systems and Designed Systems. ", "path":u"/d"},
            {"title":u"Network", "content":u"a system of lines or channels resembling a network ", "path":u"/e"},
            {"title":u"Network", "content":u"to distribute for broadcast on a television network", "path":u"/f"}]

# Add each document to index
# Opening the index writer inside a with statement causes it to automatically close/commit when it finishes
with ix.writer() as writer: 
    for doc in documents:                                         
        writer.add_document(title = str(doc["title"]), content = str(doc["content"]), path = str(doc["path"])) 

### Search
As we have defined the schema and indexed the documents, now we can search over the index with a query to retrieve matching documents. Each document is ranked according to a scoring function. There are a few types of scoring functions supported by Whoosh like term frequency, TF-IDF, Cosine ranking, BM-25, Hiemstra LM probabilistic weighting algorithm, DFree probabilistic weighting algorithm etc. The default scoring function implemented by Whoosh is BM25. By default, the Whoosh searcher only looks through one field at a time and uses 'AND' logic to combine tokens within the search query (ie. searching for "system example" is like searching for 
"'system' AND 'example'" instead of "'system' OR 'example' OR 'system example'").

We can call **_searcher()_** on the Index object to get a Searcher object. The searcher is opened using a with statement so that the searcher is automatically closed when we’re done with it.  We use the **_search()_** method, which takes a **_Query_** object and returns a **_Results_** object to query the index. By default the results contains at most the first 10 matching documents. To get more results, we can use the **_limit=count_** flag. To get all the results, use **_limit=None_** flag

A query parser is used to convert a query string submitted by a user into query objects. The default parser is the QueryParser object which takes the name of the default field to search and the schema of the index we want to  search on as arguments. In the below example, the default field on which we are searching is **_content_**. Once we have the QueryParser object, we call **_parse()_** on it to parse the query string into a query object.

If we don't explicitly specify AND or OR clauses, by default Whoosh treats the words as if they were connected by AND, meaning all the terms must be present for a document to match. In the example, the search was performed as 'system' AND 'example' from the indexed documents due to which only one document was matched by the searcher. 

In [25]:
from whoosh.qparser import QueryParser

with ix.searcher() as searcher:
    user_query = QueryParser("content", ix.schema).parse("system example")
    results = searcher.search(user_query)
    print(results)
    print("-----------------------------------------------------------------------------------------------------------------")
    print(results[0:1])

<Top 1 Results for And([Term('content', 'system'), Term('content', 'example')]) runtime=0.0008163250004145084>
-----------------------------------------------------------------------------------------------------------------
[<Hit {'content': 'An example of a system is all the organs that work together for digestion', 'path': '/c', 'title': 'System example'}>]


### Search using OR
To change the parser to use OR instead, so that any of the terms may be present for a document to match, we can configure the QueryParser using the **_group_** keyword argument. The Or query lets documents that contain more of the query terms score higher. In our case, a document with four occurances of system would normally out-score a document that contained one occurance each of system and example. However, users usually expect documents that contain more of the words they searched for to score higher. For this functionality, we can use the factory() class method of OrGroup and pass a value between 0 and 1 as the scaling factor.

We can use the **_terms=True_** keyword argument to **_search()_** to have the search record which terms in the query matched which documents.

In [26]:
from whoosh.qparser import OrGroup
with ix.searcher() as searcher:
    og = OrGroup.factory(0.9)
    user_query = QueryParser("content", ix.schema, group = og).parse("system example")
    results = searcher.search(user_query, terms=True)
    print(results)
    print("-----------------------------------------------------------------------------------------------------------------")
    
    i = 1
    for r in results:
        print("Result", i, ": ", r)
        i += 1

<Top 4 Results for Or([Term('content', 'system'), Term('content', 'example')]) runtime=0.0009111469998970279>
-----------------------------------------------------------------------------------------------------------------
Result 1 :  <Hit {'content': 'An example of a system is all the organs that work together for digestion', 'path': '/c', 'title': 'System example'}>
Result 2 :  <Hit {'content': 'This is an example for whoosh search', 'path': '/a', 'title': 'Example'}>
Result 3 :  <Hit {'content': 'a system of lines or channels resembling a network ', 'path': '/e', 'title': 'Network'}>
Result 4 :  <Hit {'content': 'A system is a group of interacting or interrelated entities that form a unified whole', 'path': '/b', 'title': 'System'}>


### Search multiple fields
In some cases, we might want to search on multiple fields, like searching on both the title and content fields. We can use a **_MultifieldParser_** for those cases. This is just like the normal **_QueryParser_**, but instead of a default field name string, the **_MultifieldParser_** takes a sequence of field names.

When we search on both title and content fields below, we can see that the order of Results 3 and 4 have interchanged.

Whoosh also provides functions to display the terms in the documents in the result list that matched the query along with the field names.

In [27]:
from whoosh.qparser import MultifieldParser

fields = ["title", "content"]

with ix.searcher() as searcher:
    og = OrGroup.factory(0.9)
    user_query = MultifieldParser(fields, ix.schema, group = og).parse("system example")
    results = searcher.search(user_query, terms=True)
    print(results)
    print("-----------------------------------------------------------------------------------------------------------------")
    
    i = 1
    for r in results:
        print("Result", i, ": ", r)
        print("Score: ", r.score)
        # If the results object was created with terms=true, it can display the terms that matched the query
        if results.has_matched_terms():
           # print the terms that matched in each result along with the fields
            print("Matched terms:", r.matched_terms())
        print()
        i += 1

<Top 4 Results for Or([Term('title', 'system'), Term('content', 'system'), Term('title', 'example'), Term('content', 'example')]) runtime=0.0011054209999201703>
-----------------------------------------------------------------------------------------------------------------
Result 1 :  <Hit {'content': 'An example of a system is all the organs that work together for digestion', 'path': '/c', 'title': 'System example'}>
Score:  5.443909911712837
Matched terms: [('title', b'system'), ('title', b'example'), ('content', b'example'), ('content', b'system')]

Result 2 :  <Hit {'content': 'This is an example for whoosh search', 'path': '/a', 'title': 'Example'}>
Score:  3.5409093744901012
Matched terms: [('title', b'example'), ('content', b'example')]

Result 3 :  <Hit {'content': 'A system is a group of interacting or interrelated entities that form a unified whole', 'path': '/b', 'title': 'System'}>
Score:  2.8712141532598934
Matched terms: [('title', b'system'), ('content', b'system')]

Re

## Part 2: Expanding our horizons with Whoosh
So far we have seen the basic functionalities offered by Whoosh and tested them on a small corpus of six documents. To understand more deeply, now we'll use the dataset provided for Homework 1 from Quizlet (https://quizlet.com), to create the index and demonstrate search and a few other functionalities offered by Whoosh. I have modified the dataset to include a created on date for each document(the code for which is given at the end of this notebook). The entity name is treated as title, the definition as the content of the documents, the document id as id and the created on date as createdDate field in the schema of the below example. 

In [28]:
import re
def createIndex(file):
    # Define schema
    schema = Schema(title=TEXT(stored=True, field_boost=2.0), content=TEXT(stored = True), id=ID, createdDate = DATETIME(stored = True))

    # Check if indexDocs exists and create it if it doesn't
    if not os.path.exists("indexDocs"):
        os.mkdir("indexDocs")

    # Create the index using the specified schema
    ix = index.create_in("indexDocs", schema)

    documents = []
    # open dataset and parse documents  
    with open(file, 'r') as f:
        for line in f:
            entity, doc_id, text, date, time = re.findall(r'(.*)[ \t](\d*)[ \t](.*)[ \t](.*)[ \t](.*)', str(line))[0]
            documents.append({"title": entity, "content": text, "id": doc_id, "date":date})

    # Add each document to index
    # Opening the index writer inside a with statement causes it to automatically close/commit when it finishes
    with ix.writer() as writer: 
        for doc in documents:                                         
            writer.add_document(title = str(doc["title"]), content = str(doc["content"]), id = str(doc["id"]), createdDate = doc["date"]) 

file = "dataWithDates.txt"
createIndex(file)

### Results
Whoosh provides many functions to filter the results based on keywords, get more results similar to a particular result, get highlighted snippets from the document containing the search terms and many more. 

To the function below, we pass a query to search on. Since many documents in our dataset have the same title, we are searching only on the contents of the documents. So, we have used **_QueryParser_** and searching using OR. We are limiting the number of search results to 5. 

We can get the total count of matched documents by calling len() function on results but for very indexes this can take a long time. So, Whoosh provides two functions to get the minimum and maximum length estimates for all results.

In [29]:
from whoosh.index import open_dir
from whoosh.query import DateRange
from datetime import datetime, timedelta

def getResults(queryText, parser, filterQuery=False):

    ix = open_dir("indexDocs")
    
    with ix.searcher() as searcher:
        # parse the query
        user_query = parser.parse(queryText)
        print(user_query)
        # if filterQuery flag is set, filter out results based on their created date
        if filterQuery:
            # filter queries older than 2 years and limit search results to 5
            filterq = DateRange("createdDate", None, datetime.now() - timedelta(days=730))
            result = searcher.search(user_query, terms=True, limit=5, mask=filterq)
        else:
            result = searcher.search(user_query, terms=True, limit=5)

        print(result)
        print("-----------------------------------------------------------------------------------------------------------------")
        print("Query: ", queryText)
        i = 1
        for r in result:
            print("Result", i)
            print("Score: ", r.score)
            print("Entity: ", r['title'])
            print("Document: ", r["content"])
            print("Created on:", r["createdDate"])
            print()
            i += 1

        print("-----------------------------------------------------------------------------------------------------------------")
        found = result.scored_length()
        print("Total number of matched documents: ", len(result))
        
        low = result.estimated_min_length()
        high = result.estimated_length()

        print("Scored", found, "of between", low, "and", high, "documents") 

In [30]:
og = OrGroup.factory(0.9)
parser = QueryParser("content", ix.schema, group = og)
getResults("relational database", parser)

(content:relational OR content:database)
<Top 5 Results for Or([Term('content', 'relational'), Term('content', 'database')]) runtime=0.026040249999823573>
-----------------------------------------------------------------------------------------------------------------
Query:  relational database
Result 1
Score:  7.595765653604991
Entity:  relational database
Document:  a database built using the relational database model
Created on: 2017-11-28

Result 2
Score:  7.5030606580811625
Entity:  relational database
Document:  relational database schema with data
Created on: 2018-05-17

Result 3
Score:  7.165325947928404
Entity:  relational algebra
Document:  - theoretical way of manipulating a relational database
Created on: 2018-06-13

Result 4
Score:  7.165325947928404
Entity:  relational database
Document:  a database using the relational data model.
Created on: 2019-03-04

Result 5
Score:  6.858305105226439
Entity:  relational algebra
Document:  relational database model based on this typ

### Fuzzy Search
Whoosh provides a **_FuzzyTermPlugin_** which enables fuzzy search on documents. The query will match any similar term within a certain number of "edits". We add the **_FuzzyTermPlugin_** to the parser to enable fuzzy search.

In [31]:
from whoosh.qparser import FuzzyTermPlugin

og = OrGroup.factory(0.9)
parser = QueryParser("content", ix.schema, group=og)
parser.add_plugin(FuzzyTermPlugin())
getResults("garbage collect~", parser)

(content:garbage OR content:collect~)
<Top 5 Results for Or([Term('content', 'garbage'), FuzzyTerm('content', 'collect', boost=1.000000, maxdist=1, prefixlength=0)]) runtime=0.03206239200062555>
-----------------------------------------------------------------------------------------------------------------
Query:  garbage collect~
Result 1
Score:  9.256211144678884
Entity:  garbage collection
Document:  there is a third phase of 2pc called garbage collection. replicas must retain records of past transactions, just in case leader fails. in practice, leader periodically tells replicas to garbage collect.
Created on: 2018-03-29

Result 2
Score:  8.581261883678883
Entity:  application layer
Document:  this collects and disseminates data e.g. a user interface needs to collect data in a standard way i.e. through a keyboard
Created on: 2020-01-01

Result 3
Score:  8.092583741208884
Entity:  data mining	9912	sophisticated statistical analysis designed to discover unhypothesized relationships 

### Filter Search Results
You can filter out results based on any criteria using Whoosh. In the below example, we are filtering out results that were created over two years ago. **_getResults()_** is called with filterQuery enabled and disabled to show the difference in the search results. In the results of the first search, that filters out results older than two years, we can see that the created date of each of the results is in the past two years. And the results of the second search includes results that are older than two years.

In [32]:
og = OrGroup.factory(0.9)
parser = QueryParser("content", ix.schema, group=og)

getResults('relational database', parser, filterQuery=True)
print("\n\n")
print("Search without filtering")
print("-----------------------------------------------------------------------------------------------------------------")
getResults('relational database', parser)

(content:relational OR content:database)
<Top 5 Results for Or([Term('content', 'relational'), Term('content', 'database')]) runtime=0.333525453999755>
-----------------------------------------------------------------------------------------------------------------
Query:  relational database
Result 1
Score:  7.5030606580811625
Entity:  relational database
Document:  relational database schema with data
Created on: 2018-05-17

Result 2
Score:  7.165325947928404
Entity:  relational algebra
Document:  - theoretical way of manipulating a relational database
Created on: 2018-06-13

Result 3
Score:  7.165325947928404
Entity:  relational database
Document:  a database using the relational data model.
Created on: 2019-03-04

Result 4
Score:  6.858305105226439
Entity:  relational database
Document:  a database built using the relational data model
Created on: 2020-01-24

Result 5
Score:  6.655912039200134
Entity:  data integrity
Document:  in relational database, a condition of data in a datab

### “Did you mean... ?” Correcting errors in user queries
Whoosh can quickly suggest replacements for mis-typed words by returning a list of words from the index that are close to the mis-typed word. The two main strategies used by Whoosh for correcting words are using the terms from the index and using the words from a word list. 

In [33]:
queryText = "garba collect"

og = OrGroup.factory(0.9)
parser = QueryParser("title", ix.schema, group=og)
query = parser.parse(queryText)

ix = open_dir("indexDocs")

with ix.searcher() as searcher:
    corrected = searcher.correct_query(query, queryText)
    if corrected.query != query:
        print("Did you mean:", corrected.string)

Did you mean: garbage collect


The functionalities described above are only a few of the features provided by Whoosh. Whoosh offers a lot more features for searching which can be found at https://whoosh.readthedocs.io/en/latest/.

### Code to add created dates to the dataset

In [21]:
import numpy as np
import pandas as pd

# define a function to generate dates
def random_dates(start, end, n, unit='D', seed=None):
    if not seed:
        np.random.seed(0)

    ndays = (end - start).days + 1
    return pd.to_timedelta(np.random.rand(n) * ndays, unit=unit) + start

np.random.seed(0)
start = pd.to_datetime('2015-01-01')
end = pd.to_datetime('2018-01-01')
# get a list of random dates
dates = random_dates(start, end, 30917)

docs = []
i = 0
with open('homework_1_data.txt', 'r') as f:
    for line in f:
        line = line[0:len(line)-2]
        docs.append(line + "\t" + str(dates[i]) + "\n")
        i += 1
        
# create a file with dates appended to each line
with open('homework_1_data_dates.txt', 'w') as f:  
    f.writelines(docs)